In [ ]:
# Standard libraries
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from lib.gallegosalas.data_fc_db_sessional import DataFCDatabase
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.signals import zscore

%load_ext autoreload
%autoreload 2

In [ ]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata2/'

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
dataDB.read_neuro_files()

In [ ]:
dataDB.baseline_normalization()

In [ ]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.dataNeuronal[0].shape[1])

In [ ]:
dataDB.calc_shortest_distances()

fig, ax = plt.subplots(ncols=2)
ax[0].imshow(dataDB.allenMap)
ax[1].imshow(dataDB.allenDist)
plt.show()

len(dataDB.allenCounts)

In [ ]:
dataDB.channelLabels

In [ ]:
nRows = len(dataDB.sessions)
nCols = dataDB.get_n_data_types()

for iMouse, mousename in enumerate(dataDB.mice):
    fig, ax = plt.subplots(nrows=nRows, ncols=nCols, figsize=(4*nCols, 4*nRows), tight_layout=True)
#     fig.suptitle(mousename)
    
    for iSession, session in enumerate(dataDB.sessions):
        ax[iSession, 0].set_ylabel(session)
        
        for iDataType, datatype in enumerate(dataDB.get_data_types()):
            if iSession == 0:
                ax[0, iDataType].set_title(datatype)
                
            mouseRows = dataDB.get_rows('neuro', {'mousename' : mousename, 'session' : session, 'datatype' : datatype})
        
            for idx, row in mouseRows.iterrows():
                data = np.copy(dataDB.dataNeuronal[idx])
                nTrial, nTime, nChannel = data.shape
                times = dataDB.get_times(nTime)

                ax[iSession, iDataType].plot(times, np.mean(data, axis=0))
                dataDB.label_plot_timestamps(ax[iSession, iDataType], linecolor='r', textcolor='pink')
        
plt.show()

In [ ]:
idxTrial = dataDB.get_rows('neuro', {'session' : '20180301a', 'datatype' : 'dff_trial'}).index[0]
idxSession = dataDB.get_rows('neuro', {'session' : '20180301a', 'datatype' : 'dff_session'}).index[0]

In [ ]:
dataTrial = dataDB.dataNeuronal[idxTrial]
dataSession = dataDB.dataNeuronal[idxSession]

plt.figure()
plt.plot(dataTrial[0, :, 2])
plt.plot(dataSession[0, :, 2])
plt.show()